In [5]:
import random
import re
import os.path
import pickle
import copy
import sys

import pandas as pd
import numpy as np
import hashlib
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup
from random import choice
from string import ascii_uppercase
from string import digits
import sys
import time

import re
import string

from threading import Thread #Support Multi-Threading Crawling

import openpyxl
from openpyxl import load_workbook
import xlsxwriter 

In [6]:
### All the used Taxonomy ###

taxonomy_name = 'github_brand domain_12022020'

In [7]:
taxonomy_loc = 'C:\Domain Keyword Auditing\{}.xlsx'.format(taxonomy_name)
original_df = pd.read_excel(taxonomy_loc,sheet_name='input')
original_df = original_df.fillna("")

In [8]:
### Edit column name based on the Taxonomy type to fit the column name script use ###

if 'generic' in taxonomy_name:
    original_df = original_df.rename(columns={"product":"brand"})
    original_df['category'] = ''

In [1]:
df = copy.deepcopy(original_df)
df

# [C] 1. Preparing the Domain keyword data

In [10]:
### Function for dedup a list and but keep its original sequence ###

def unique(sequence):
    seen = set()
    # Using set to keep track of the unique vlaue 
    return [x for x in sequence if not (x in seen or seen.add(x))] 

In [11]:
# Function for cleaning the brand. Also clean the title, url from the search result website (remove all non-char)
# eg. 'Rue89 - Home | Facebook https://www.facebook.com/Rue89' ---> ' rue89 home facebook https en gb facebook com rue89 '

def source_cleaner(title):
    pattern = re.compile('[^a-zA-Z0-9\s]')  #An Regex pattern stands for all the upper and lower case letter and numbers, ^ means not
    char_title_lst = pattern.sub(' ', title).split(' ')  # Substitute all elements which not number or char using ' '

    char_title_lst = [x for x in char_title_lst if x != '']  # Getting all the not ' ' element from the list  
    char_title = ' '.join(char_title_lst).lower()
    
    return(char_title)   #return cleaned title

In [12]:
### Function: turning the brand into a list of brands (prepare for matching to crawled website text)

def to_brand_list(brand_name):
    
    ### Function for trim the text ###
    def trim(text):
        new_text = ' '.join([t for t in text.split(" ") if t != ''])
        return(new_text)
    
    ### work on brand names ###
    brand_name_lst = brand_name.split(',')  # Just in case if multiple brand name is given like "verizon" and "verizon wireless"
    brand_name_lst = [trim(b) for b in brand_name_lst]
    
    brand_lst = []
    
    for brand in brand_name_lst:
        
        # 1. Originl form
        # If you want to find a word within sentence, there is space around the wor. (eg. 'spotify' -> '(space) spotify (space)')
        brand_lst.append(' {} '.format(brand))   
        
        # 2. Space removed form
        if " " in brand:
            brand_lst.append(' {} '.format(brand.replace(' ','')))  
        
        # 3. Special Char removed form
        cleaned_brand = source_cleaner(brand)
        if cleaned_brand != brand:
            brand_lst.append(' {} '.format(cleaned_brand))
        
        # 4. Special Char & space removed form
        if " " in cleaned_brand:
            brand_lst.append(' {} '.format(cleaned_brand.replace(' ','')))  
            
    brand_lst = unique(brand_lst)
    
    return(brand_lst)


In [13]:
### Function: getting "cleaned domain" (without tail), lowered brand-term, "brand_list" for matching ###

def to_new_df(df):
    
    # lower case the dataframe column name
    df.columns = map(str.lower,df.columns)
   
    # Getting domain with or without tail removed (Based on need)
    clean_domain = []
    for i in range(len(df)):
        domain = df['domain'][i]
        clean_str = domain.lower()   # 1. New method: only change domain to lower case, not remove tail when search
        clean_domain.append(clean_str)
    df['clean_domain'] = clean_domain

    # Clean the brand-term (remove extra space caused by manually input)    
    # Because 'space' will be used as a important judgement in function to_brand_list, we couldn't allow misinputed 'space'
    # eg. " hotel.com   " -> "hotel.com"
    
    # Some brand will give multiple name like verizon, "verizon" and "verizon wireless"
    df['brand'] = df['brand'].apply(lambda x : ','.join([" ".join([each for each in b.split(' ') if each != '']) for b in x.split(',')]))
    df['brand_low'] = df['brand'].str.lower()  # Turn a whole column into lower    
    
    # Getting brand_list for matching
    df['brand_list'] = df['brand_low'].apply(to_brand_list)
    
    # Some brand will give multiple name like verizon, "verizon" and "verizon wireless"
    df['category'] = df['category'].apply(lambda x : ' '.join([each for each in x.split(' ') if each != '']))
    df['category'] = df['category'].str.lower()  # Turn a whole column into lower 

    # Helper column will be used when matching later
    df['judgement_brand'] = ''
    df['match_brand'] = ''
    df['match_location_brand'] = ''
    df['search_result'] = ''
    df['search_result_raw'] = ''
    

In [2]:
# Reformat the dataframe
to_new_df(df)

df

# [C] 2. Function for getting Hiding Identity

In [19]:
chrome_ua = pd.read_excel(r'C:\Anti Blocking.xlsx','chrome_ua')
user_agent_list = list(chrome_ua[chrome_ua['os'] == 'Windows']['user agent'])  #All Windows OS on Chrome_ua
user_agent_list[:10]

['Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.99 Safari/533.4',
 'Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/10.0.649.0 Safari/534.17',
 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/10.0.649.0 Safari/534.17',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.83 Safari/537.1',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.152 Safari/537.22',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.31 (KHTML, like Gecko) Chrome/26.0.1410.64 Safari/537.31',
 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36 HubSpot Webcrawler - web-crawlers@hubspot.com',
 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.63 Safari/537.36',
 'Mozilla/5.0 (Wi

In [20]:
### Function for random sleep ###

def random_sleep(assign=False):
    ticket = random.choice(list(range(10)))  # lucky number determine sleep or not
    
    if assign == False:
        i = random.choice(list(range(1,3)))   # Random sleep time
    else:
        i = assign

    if ticket in (0,8): #The number that will triger
        print("\nSleep for {} seconds\n".format(i))
        time.sleep(i)

# [C] 3. Function for crawling

In [21]:
### Function (Layer 4) Simulate information for call Bing API ###

def get_sim_info():
    
    ### Simulate cv number to pass into API ###
    letter_n_number = ascii_uppercase + digits + digits
    sim_cvid = ''.join(choice(letter_n_number) for i in range(32))
    
    ### Simulate form number ###
    form_list = ['QBRE','QBLHCN']
    sim_form = choice(form_list)
    
    return(sim_cvid,sim_form)

In [22]:
### Function (layer 4) Calculate words location based on word's index in all organics ###

def smallest_idx(judge_lst, organic_list):

    min_match_score = 100000000
    min_organic = ''
    min_loc = ''
    min_word = ''

    for brand in judge_lst:
        for i in range(len(organic_list)):

            if (min_organic == '') or (i<=min_organic):

                organic = organic_list[i]
                organic_long = len(organic)-2   # The number of text in one SERP organic result, deduct added 2 (blank) in front and back

                try:
                    match_index = organic.index(brand)+1-1  # Index of matched location. Just wanna lay this out, python index start from 0, so "+1". The sentence start with space, so "-1"
                    match_loc = round(match_index / organic_long,3)  # Text % of place brand named get match in the organic 
                    match_score = i + 1 + match_loc  # match_score = the_order_of_organic_been_match + location_matched_on_that_organic. Smaller the better

                    if match_score < min_match_score:
                        min_match_score = match_score
                        min_organic = i+1
                        min_loc = match_loc
                        min_word = brand
                except:
                    pass
    
    location = 'organic:{} | location:{}'.format(min_organic,min_loc)
    
    return(min_organic, location, min_word)

In [23]:
### Function (layer 3) for getting source from XML for a query ###

def get_source_for_query(query):
    
    ### Get simulated url ###
    sim_cvid, sim_form = get_sim_info()
    url = 'https://www.bing.com/search?format=rss&q={qr}&qs=n&form={f}&cvid={cid}'.format(qr=query,f=sim_form,cid=sim_cvid)
    
    ### Get a new agent for crawling ###
    agent = random.choice(user_agent_list) 
    headers = {'User-Agent': agent}
    
    ### Get the XML source of certain query ###
    response = requests.get(url, headers=headers, timeout = 100) 
    soup = BeautifulSoup(response.text, "lxml")
    
    ### Get source for all organics ###
    all_source = soup.findAll("item")
    
    return(all_source)

In [24]:
### Function (layer 3) for Combining all the title strings, URL and description string from the XML file ###

def source_to_str(title_source):
    
    all_organic_raw = []  # 1. Get a list of original organic title + url from source
    title_len = len(title_source)
    
    for i in range(title_len): 
        
        ### Initialize ###
        each = title_source[i]
        organic_raw = ''
        
        ###### Get title for organic ######
        
        # a. Find title
        title_raw = each.find("title").text.replace('<title>','').replace('</title>','')
        
        # b. Add it to organic_raw string
        if title_raw != '':
            organic_raw = ' ' + title_raw.lower() + ' '

        ###### Get url for organic ######
        url_raw = ''
        
        # a. Try <link>...</link>
        url_raw = each.find("link").text.replace('<link>','').replace('</link>','')
        
        # b. If <link>...</link> couldn't be found, try with Regex 
        if url_raw == '':
            url_raw = re.search("<link/>.*<description>",str(each)).group().replace('<link/>','').replace('<description>','')
        
        # b. Add it to organic_raw string
        if url_raw != '':
            organic_raw = organic_raw + url_raw.lower() + ' '
            
        ###### Grt description for organic ######
        
        # a. Find description
        desc_raw = each.find("description").text.replace('<description>','').replace('</description>','')
        
        # b. Add it to organic_raw string
        if desc_raw != '':
            organic_raw = organic_raw + desc_raw.lower() + ' '
        
        
        ### Appending each organic information to a list ###
        all_organic_raw.append(organic_raw)
    
    ### Creating 2 another version of organic result ###
    all_organic = [' ' + source_cleaner(x) + ' ' for x in all_organic_raw]  # 2. cleaned version of a list of all organics
    all_str = ' '.join(all_organic)  # 3. combine all orgnaics together to 1 string *** Note: Adding blank between organics, make it 3 block between organics, avoid miss match due to front and end connected 

    
    ### Summary ###
    # all_str = a string contains all organics information. Will used for matching
    # all_organic = a list of organics. Each one in the list is string that has both title + url. The string is cleaned
    # all_organic_raw = a list of organics, but not cleaned
    
    return(all_str, all_organic, all_organic_raw)

In [25]:
### Function (layer 3) for matching brand name in the organic string ###

def brand_name_match(domain_brand_list, source_str, organic_list):
    
    # compare search result to original brand name
    judge_lst = [ele for ele in domain_brand_list if (ele in source_str)] # if the original brand name in crawled titles
    judge = judge_lst != []
    
    # Getting the smallest index' percentage to the whole string among the match keywords from brand_list
    match_location = ''
    match_word = ''
    
    # Judge true means we could find brand in the SERP page, and the domain have relevancy with the original brand-term
    if judge is True:
        match_organic_order, match_location, match_word = smallest_idx(judge_lst, organic_list)
        
    # organic_requirement means the brand have to be in the first 5 organic result
    organic_requirement = 5
    record_number = 1 if judge is True and match_organic_order <= organic_requirement else 0
    
    return(record_number, match_word, match_location)

In [26]:
### Function (layer 2) for getting crawled information, calculate matched location, and load into dataframe ###
# After make sure its a website page with result #

def crawl_calc_load(df, index):
    
    ### Initialize ###
    row = df.iloc[index]
    domain_name = row['clean_domain']
    brand_name = row['brand_low']
    domain_brand_list = row['brand_list']
    similar_domain_idx = list(df[(df['clean_domain'] == domain_name) & (df['brand_low'] == brand_name)].index)
    
    ### Getting the crawled result if available, try N trails to avoid data lost due to IP-rotating ###
    
    ## ******************** Note: Why doing this N trails ******************** ##
    ## Seems that Threading  will case request failure some time               ##
    ## Need to try N times of request to confirm if organics really not exist  ##
    ## *********************************************************************** ##
    
    source_str = ''
    count = 0
    trails = 10
    while source_str == '' and count<trails:
        title_source = get_source_for_query(domain_name)   # Getting all organic blocks
        source_str, organic_list, organic_raw_list = source_to_str(title_source)  # 1.Source_str contains all title and url string on that page 2.Organic_list contains all title+url string by order
        count += 1
        
        if source_str == "":
            print("SOUP-PROBLEM :: Tried {} {}/{} times".format(domain_name,count,trails))
        
    ### Match the brand name to the orgnaic string ###
    record_number, match_word, match_location = brand_name_match(domain_brand_list, source_str, organic_list)
    
    ### if no organic exists; record 'no orgnaic' to dataframe and pass parameter signal as 'nothing' ###
    signal = 'nothing'
    if source_str == '':
        signal = 'does not have organic result'
        
        organic_list = 'no organic result'
        organic_raw_list = ''
        
    
    ### Updating the judgement value (1 means relevant, 0 means no relevancy) for domain with same domain_cleaned value ###
    for index in similar_domain_idx: 
        df.at[index,'judgement_brand'] = record_number  
        df.at[index,'search_result'] = organic_list
        df.at[index,'search_result_raw'] = organic_raw_list
        df.at[index,'match_brand'] = match_word
        df.at[index,'match_location_brand'] = match_location
    
    return(signal)

In [27]:
### Function (layer 1) for getting the crawling result ###

def get_result(df, index):
    
    ### Try to send request and connect to Bing API ###
    counter = 0 #Giving more chance to connect
    count_end_threshold = 10 #When the loop end
    
    while counter <= count_end_threshold:   #Trying to open an API for 10 times, if all failed, pass
        try:
            signal = crawl_calc_load(df, index)
            return('success',signal)
        except (ConnectionError, requests.exceptions.SSLError, 
                requests.exceptions.ProxyError, ConnectionResetError, AttributeError) as err:
            print("Handling run-time error: {}".format(err))
            if counter >= count_end_threshold:
                    print("\ncan't connect to API, will go out loop and try again\n")
                    return('fail',signal)
            pass
        counter += 1   

# [C] 4.1 Crawling process

In [28]:
### Copying the dataframe, and prepare for crawling ###


df_final = copy.deepcopy(df)



## :: 1.1 Multi-Threading Crawling ::

In [29]:
### ****** Function (layer 0) for gett results for all domains [Support Multi-Threading Crawling] ****** ###

def get_result_for_all(df_final,i):
    
    df_len = len(df_final)
    
    judgement_value = df_final.iloc[i]['judgement_brand'] 
    if judgement_value == '':    # Didn't make any relevancy judgement yet 
        ### Getting the crawled data and update the renew_count. Renew a driver if we can't get result about a domain
        result = 'fail'
        signal = 'nothing' #signal that if orgnaics exist on XML file page
        while result == 'fail':
            result,signal = get_result(df_final, i)  

        ### If no organics on a page ###
        if signal == 'does not have organic result':    
            dm = df_final.iloc[i]['clean_domain']
            print('{}/{} NO ORGANIC FOUND FOR {}'.format(i, df_len-1, dm))
        elif signal == 'nothing':
            print('{}/{} crawled'.format(i, df_len-1))

        ### Implement random sleep after did one crawling
        random_sleep(1)

    else: # If the judgement made already, go to the next domain
        print('{}/{} already have result'.format(i, df_len-1))

In [5]:
df_len = len(df_final)
df_len

In [4]:
### Multi-Threading Crawling ###


start = datetime.now()  # record start time 

threadlist = []
try:
    for i in range(df_len):
        t = Thread(target=get_result_for_all,args=(df_final,i,))
        t.start()
        threadlist.append(t)

    for b in threadlist:
        b.join()
except KeyboardInterrupt:
    print("Unexpected error:", sys.exc_info())
    pass

# Print that crawl is finished
print("\nCrawling finished\n")

end = datetime.now() # record end time 
print("\n", end - start)

In [6]:
### Double Check ###

df_final[df_final['search_result_raw'] == '']

In [8]:

### Make a copy of df_final
df_final_out = copy.deepcopy(df_final)
df_final_out

# [M] 1. Brand URL Matching

## :: 1. Creating empty Brand+GEO ::

In [35]:
### Function (1nd step) for get empty dataframe to store organics for brand+geos ###
# max_match means also search (pure geo) like github.fr

def get_official_pre(brands,max_match=0):
        
    ### Get the list of countries ###
    geo_file_loc = r'C:\A_Workplace\Projects\Domain Cleaning\Useful\geo_file.xlsx'
    geo_list = list(pd.read_excel(geo_file_loc,'geo')['geo_list'])
    
    ### Get the Pure Short Geo list ###
    if max_match == 1:
        short_geo_list = ["{} (pure)".format(x) for x in geo_list if len(x)==2]
        geo_list = geo_list + short_geo_list

    # Get the number of brands and Geos
    numOfBrands = len(brands)
    numOfGeo = len(geo_list)
    # Geo and brand lists will be used to build official_df_pre dataframe
    col_geos = geo_list * numOfBrands
    col_brands = sorted(brands * numOfGeo)

    # Build the official_df_pre dataframe
    official_df_pre = pd.DataFrame({'brand':col_brands, 'geo':col_geos,'organics':''})
    return(official_df_pre)

In [36]:




###### get EMPTY official_df_pre !!! (Be careful if official_df_pre has result) ######

official_df_pre = get_official_pre(brands,0)
print('{} "brand+geo" need to get'.format(len(official_df_pre)))





58 "brand+geo" need to get


## :: 2.1 Multi-Threading Crawling ::

In [38]:
###### Function (1st step + layer 2) Getting official URL for a brand ######
# cat: assign category to a brand for getting its official site
# select_organic_num: treat how many TOP organics as the brand's official information

def get_official(brand, geo, cat=0, select_organic_num=3):
    
    ### Pre-Process ###
    brand_list = brand.split(',')
    
    ### If no value assigned to the parameter "cat", get the category value from dataframe ###
    if cat == 0:
        cat = df_final[df_final['brand_low'] == brand]['category'].unique()[0]  # Category of the brand
    
    off_organics = []

    for each in brand_list:   # For each brand form "verizon" and "verizonwireless"
        
        ### For Geo's full name and short name, we use different searching strategy ###
        geo_length = len(geo)
        if geo_length == 2:
            brand_search = "{}.{} {}".format(each, geo, cat)  # short country_name + category
        elif '(pure)' in geo:
            brand_search = "{}.{}".format(each, geo.replace('(pure)',''))  # short country_name only
        else:
            brand_search = "{}.com {} {}".format(each, geo, cat) # long country_name + category
        
        ### Try N Times to get brand+geo related organics ###
        brand_search_source = []
        count = 0
        trails = 10
        while brand_search_source == [] and count<trails:
            brand_search_source = get_source_for_query(brand_search)  # Get the XML source
            count += 1
            if brand_search_source == []:
                print("SOUP-PROBLEM :: Tried {} {}/{} times".format(each+"+"+geo,count,trails))
                     
        ### Get the organic results for the brand+geo ###
        source_str, organic_list, organic_raw_list = source_to_str(brand_search_source)  # Get all organic info
        
        ### Append the result ###
        for each in organic_raw_list[0:select_organic_num]:
            off_organics.append(each)
    
    ### Dedup the off_organics list ###
    off_organics  = list(set(off_organics))
    
    ### Flag if there is no organic result ###
    flag = 'does not have organic result' if off_organics == [] else 'nothing'
    
    return(off_organics,flag) 

In [39]:
### Function (2st step + layer 1), get offical organics for all brands [Support Multi-Threading Crawling]###

def get_all_official (official_df_pre,i,organic_num=1):
    
    num = len(official_df_pre)

    ### Judgement value help us see if we have the data we want already ###
    off_data_judge = official_df_pre.iloc[i]['organics'] 
    brand = official_df_pre.iloc[i]['brand']
    geo = official_df_pre.iloc[i]['geo']

    ### Crawl only when the organics is empty ###
    if off_data_judge == '':
        
        ### Try to crawl until get it, except meet special error ###
        continue_judge = 1
        while continue_judge == 1:
            try:
                off_organics,theFlag = get_official(brand, geo, cat=0, select_organic_num=organic_num)  # cat=0 means the function will find category form dataframe
                official_df_pre.at[i,'organics'] = off_organics
                
                ### Flag it if there is not result after several try ###
                if theFlag == 'nothing':
                    print('{}/{} {} {}'.format(i+1,num,brand,geo))  
                elif theFlag == 'does not have organic result':
                    print('{}/{} NO ORGANIC FOUND FOR {} {}'.format(i+1,num,brand,geo))
                
                random_sleep(1) # After crawl each brand, random sleep a while, assign 3 seconds sleep
                continue_judge = 0
            except (ConnectionError, requests.exceptions.SSLError, 
                    requests.exceptions.ProxyError, ConnectionResetError, AttributeError) as err:
                print("Handling run-time error: {}".format(err))
                pass
    else:
            print('{}/{} {} {} already have result'.format(i+1,num,brand,geo))

In [9]:
### Multi-Threading Crawling ###

start = datetime.now()  # record start time 

df_long = len(official_df_pre)
threadlist = []

try:
    for i in range(df_long ):
        t = Thread(target=get_all_official,args=(official_df_pre, i, 2,))
        t.start()
        threadlist.append(t)

    for b in threadlist:
        b.join()
        
except KeyboardInterrupt:
    print("Unexpected error:", sys.exc_info())
    pass

print("\nCrawling finished\n")


end = datetime.now() # record end time 
print("\n", end - start)

In [10]:
official_df_pre_out = copy.deepcopy(official_df_pre)
official_df_pre_out.tail(5)

## :: 3. Organize and Clean crawled organics ::

In [42]:
### Function (3rd step) to organize official_df_pre to official_df final version ###

def pre_to_final(df_pre):
    
    #Creating an organized dataframe
    df = pd.DataFrame({'brand':brands, 'organics':''}) 
    
    # Load from df_pre to df
    for i in range(len(df)):
        
        brand = df['brand'].iloc[i]
    
        # Get all organics from df_pre
        all_organics = list(df_pre[df_pre['brand'] == brand]['organics'])
    
        # Get unique organics
        all_organics = list(set([x for each in all_organics for x in each]))
        
        # Load
        df.iloc[i]['organics'] = all_organics
        
    return(df)

In [43]:
### Function (4th step + layer 2) to extract URL link from a line of word
# eg. 'http://formation.rue89.com/' ---> 'formation rue89 com'


def extract_url (url_text):
    
    ### pattern for finding url 
    re_pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url_raw = [x[0] for x in re.findall(re_pattern, url_text)]
    
    core_list = []
    ### Clean the extra information like "http" cleaned website ###
    for each in url_raw:
        if ('https://' in each) or ('http://' in each):
            url_split = each.split('/')
            try:
                core = [x for x in url_split if '.com' in x][0]  # If splitted part has ".com" get this term
            except:
                core = each.split('/')[2] # Core part in a url, for example "https://www.riide.com/tech-specs" ---> "www.riide.com"
        elif ('https://' not in each) and ('http://' not in each) and ('www' in each):
            url_split = each.split('/')
            try:
                core = [x for x in url_split if 'www.' in x][0] # If splitted part has "www." get this term
            except:
                core = each.split('/')[0]
        else:
            core = ""
            
        core_clean = source_cleaner(core)
        core_remove = re.sub('https|http|www|','',core_clean)
        core_final = ' ' + ' '.join([each for each in core_remove.split(' ') if each != '']) + ' '
        
        if core_final != '  ':
            core_list.append(core_final)
    
    return(core_list)

In [44]:
###### Function (4th step + layer 1) Extract core official url information from organic ######

def clean_official(official):
    result = []
    for each in official:
        
        temp = extract_url (each)
        result.append(temp[0])
    
    result = list(sorted(set(result)))  # De-dup
    
    return(result)

In [45]:
### Function (5th step) Reorder the clean_url list, make the url start with brand's char listed in the front ###

def reorder_url(row):
    brand_first_letter = row['brand'][0]
    clean_url = row['clean_url']
    
    ### The url to put in the front and back ###
    front_element = [x for x in clean_url if x.startswith(" {}".format(brand_first_letter))]
    back_element = [y for y in clean_url if y not in front_element]
    
    ### Combine all the url and return ###
    out_url = front_element + back_element
    
    row['clean_url'] = out_url
    
    return(row)

In [11]:
### Get the organized final version official df ###

official_df = pre_to_final(official_df_pre)
official_df.tail(5)

In [12]:
### Getting official url's core information

official_df['clean_url'] = official_df['organics'].apply(clean_official)
official_df = official_df.apply(reorder_url,axis=1)
official_df['pick_url'] = official_df['clean_url']
official_df.head(100)

In [13]:
###### Loading the official_df to a outside Excel for cleaning and organizing ######

print("Adding Official URL to the Taxonomy: {}\n".format(taxonomy_loc))

### The Taxonomy File and the Sheet to Edit ###
workbook=load_workbook(taxonomy_loc)
psn = 'official url prep'  #psn = prep sheet name

### Try to delete the "official url prep" sheet if any
try:
    del workbook[psn]
except:
    pass

### Adding organized url sheet "official" to the Excel ###
osn = 'official'  # osn = Organized sheet name
try:
    workbook[osn]  # Check if 'official' sheet exist, if not create a new sheet
except KeyError:
    workbook.create_sheet(osn)
    
### Save the workbook edit above ###    
workbook.save(taxonomy_loc)

### Loading official url into excel ###
writer = pd.ExcelWriter(taxonomy_loc, engine='openpyxl') 
book = load_workbook(taxonomy_loc) # Check other sheet in the excel; prevent over-writing and data loss
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
official_df.to_excel(writer, sheet_name = psn, index=0) # Load the official dataframe to the excel sheet 
writer.save()
writer.close()

print('\nOfficial_df loaded into the Excel sheet successfully!\n')
print('\n*** Please move organized official_df from the TAB [Official URL Prep] to [Official] in the Taxonomy *** \n')

## :: 4. Organize or Modify the official_df dataframe ::

In [86]:
### Function for converting string item in Excel to a list of string ###

def restore_list_from_excel(string):
    trim_string = ' '.join([x for x in string.split(' ') if x != ''])  # Trim the extra space in the string 
    new_string = trim_string[1:][0:-1]  # Remove "[" and "]"
    
    splited_list = new_string.split(',')  # split the url according to the comma between them ","
    
    ### for each url, reomve "'"; then remove extra space; lastly, adding back the front and back space ###
    new_splited_list = [' ' + ' '.join([a for a in x.replace("'","").split(' ') if a!='']) + ' ' for x in splited_list]
    return(new_splited_list)

In [87]:
### Function for converting the organized final_df to a URL dataframe ###

def excel_df_to_url_df(organized_df):
    
    organized_df['organics'] = organized_df['organics'].apply(restore_list_from_excel)
    organized_df['clean_url'] = organized_df['clean_url'].apply(restore_list_from_excel)
    organized_df['pick_url'] = organized_df['pick_url'].apply(restore_list_from_excel)
    
    return(organized_df)

In [14]:
### Read the organized official url from the Taxonomy file - official sheet
print("Will read TAB [Official] from Taxonomy file: {}".format(taxonomy_loc))
organized_df = pd.read_excel(taxonomy_loc,'official')

### From the Excel df to organized url df ###
org_official_df = excel_df_to_url_df(organized_df)

In [15]:
org_official_df

## :: 4. Work on Final DataFrame ::

In [93]:
### Function (5th step) Getting URL from organic ###

def get_organic_url (organic_raw_list):
    
    organic_url = []
    for org in organic_raw_list:
        url_list = extract_url(org)
        url_list = list(set(url_list))
        organic_url.append(url_list)
        
    return(organic_url)

In [94]:
### Function (Last step) Matching brand_url to oganic url (Exact Match) ###

def url_match(row):
    organic_url = row['organic_url']  # URL from organic result
    url_list = row['url_list']  # brand's official URL
    
    ### Only match to organic url from the first 3 organics 
    num = len(organic_url)
    threshold = 3 
    
    ### Load all organic_url to a list and record which organic are they from
    o_url = []
    o_url_from = []
    
    for i in range(num):
        one_list = organic_url[i]      
        for each in one_list:
            if each not in o_url:  # Only unique URL 
                o_url.append(each)
                o_url_from.append(i)
    
    ### Matching brand_url to organic_url
    match_url = []
    match_organic = []
    
    for each in url_list:
        if each in o_url:
            idx = o_url.index(each)  # Get index of url match in combined organic url list
            which_organic = o_url_from[idx]  # Get the orgnaic order of that match url, where it comes from
            
            match_url.append(each)
            match_organic.append(which_organic+1)
    
    ### Judgement and prepare to load ###
    if match_url != []:
        
        if any(x <= threshold for x in match_organic):
            judge = 1
        else:
            judge = 0
            
        match_url = list(set(match_url))
        
        match_organic = list(set(match_organic))
        match_organic = ['{}'.format(x) for x in match_organic]
        sentence = "organic: " + ",".join(match_organic)
    else:
        judge = 0
        match_url = ""
        sentence = ""
    
    row['judgement_url'] = judge
    row['match_url'] = match_url
    row['match_location_url'] = sentence
    
    return(row)


In [95]:
### Getting URLs from organic results ###
df_final['organic_url'] = df_final['search_result_raw'].apply(get_organic_url)

In [96]:
### Adding official_url to final df###
df_final['url_list'] = df_final['brand_low'].apply(lambda b : org_official_df[org_official_df['brand'] == b].iloc[0]['pick_url'])

In [97]:
### URL Matching ###
df_final = df_final.apply(url_match, axis=1)

# [M] 2.Domain Keyword Negative Word Matching

In [99]:


###### Match negative word in domain ######



In [16]:
domain_picked = list(pd.read_excel('negative_word.xlsx',sheet_name='domain_picked')['keyword'])  # Hand picked domain filter keywords
compliance = list(pd.read_excel('negative_word.xlsx',sheet_name='compliance')['keyword'])  # Compliance given domain filter keywords

domain_safety = [str(x) for x in list(set(domain_picked + compliance))]  # Combine
domain_safety

In [101]:
###### Function for finding irrelevant words like "career", "enterprise" in domain ######

def find_irrelevant(row):
    domain = row['domain']
    irrelevant_list = [x for x in domain_safety if x in domain]
    
    if irrelevant_list != []:
        judge = 1
        out = irrelevant_list
    elif irrelevant_list == []:
        judge = 0
        out = "no irrelevant in domain"
        
    row['domain_has_negative'] = judge
    row['domain_negative'] = out
    
    return(row)

In [102]:
df_final = df_final.apply(find_irrelevant, axis=1)

# [M] 3.Organic illegal Word Matching

In [17]:
organic_picked = list(pd.read_excel('negative_word.xlsx',sheet_name='organic_picked')['keyword'])  # Hand picked domain filter keywords
compliance = list(pd.read_excel('negative_word.xlsx',sheet_name='compliance')['keyword'])  # Compliance given domain filter keywords

organic_safety = [' '+str(x)+' ' for x in list(set(organic_picked + compliance))]  # Combine
organic_safety

In [106]:
def find_illegal(row):
    
    organic_list = row['search_result']
    
    long = len(organic_list)
    threshold = 3      # Check if sensitive word appear in the first 3 organics
    
    if long > threshold:
        long = threshold
    
    # Check if there is sensitive words in first 3 organics
    sensitive_list = []
    for i in range(long):
        organic = organic_list[i]
        check = [w for w in organic_safety if w in organic]
        if check != []:
            check = [x.replace(' ','') for x in check]
            out = "organic{} ".format(i+1) + "[" + ",".join(check) + "]"
            sensitive_list.append(out)
    
    # Load result to excel
    if sensitive_list != []:
        judge = 1
    else:
        judge = 0
        sensitive_list = "no illegal"
    
    row['organic_has_negative'] = judge
    row['organic_negative'] = sensitive_list
    
    return(row)

In [107]:
df_final = df_final.apply(find_illegal, axis=1)

# [M] 4. Non-English Matching

In [110]:
def non_eng_detector(string):
    
    ### RE pattern: Don't want latin-char, number, and non-char symbols
    extra = '“”‘’!"#$%&\'()*+,-./:；;<=>?@[\\]【】^_——`{|}~\n。！，…ːʻˈ–®•→›🙏ᐅᐉߴ\s'     #Supplement list
    RE = re.compile('[^a-zA-Z0-9\W+{}]+'.format(extra))
    other = re.findall(RE, string)
    return(other)

In [111]:
def find_non_english(row):
    
    ### organic list
    organic_raw_list = row['search_result_raw']
    
    long = len(organic_raw_list)
    threshold = 3      # Check if sensitive word appear in the first 3 organics
    
    if long > threshold:
        long = threshold   # Only check first 3 organics 
    
    other_language = []
    for i in range(long):
        organic_raw = organic_raw_list[i]
        
        other = non_eng_detector(organic_raw)
        
        if other != []:
            out = "organic{} ".format(i+1) + "[" + " ".join(other) + "]"
            other_language.append(out)
            
    if other_language == []:
        other_language = "no other language"
        judge = 0
    else:
        judge = 1
        
    row['organic_has_nonEnglish'] = judge
    row['non_english'] = other_language
    
    return(row)

In [112]:
df_final = df_final.apply(find_non_english, axis=1)

# [M] 2.2 Rematch Brand_list After Crawl (If Necessary)

In [114]:
###### Fuction: Get match for brand_name, and their match location ######

def after_crawl_match(row):
    
    # 1. Basic search result information needed for both matching url & brand
    target = row['brand_list']
    search_result = row['search_result']
    result_sentence = ' '.join(row['search_result'])  # Added an extra blank between organic results (3 total) to differentiate them and avoid mismtch
    
    record_number, match_word, match_location = brand_name_match(target, result_sentence, search_result)
    
    return(record_number, match_word, match_location)

In [115]:
### Function: Loading brand match information into dataframe (Rematch after crawl) ### 

def brand_match(row):
   
    # Get url match result
    record_number, brand_match, brand_match_location = after_crawl_match(row)
    
    # Load into df
    row['judgement_brand'] = record_number  
    row['match_brand'] = brand_match
    row['match_location_brand'] = brand_match_location

    return(row)

In [116]:
# Get brand_low and brand_list again
df_final['brand_low'] = df_final['brand'].str.lower() 
df_final['brand_list'] = df_final['brand_low'].apply(to_brand_list)

# Erase original match
df_final['judgement_brand'] = ''
df_final['match_brand'] = ''
df_final['match_location_brand'] = ''

### Find brand match judgement, matched_url and their match_location ###
df_final = df_final.apply(brand_match, axis=1)

In [18]:
df_final_out = copy.deepcopy(df_final)
df_final_out.head(100)

# X1. Reorganize the final dataframe

In [119]:
### Put the columns in the dataframe into the order as the following ###

column_order = ['brand',
             'domain',
             'cluster_id',
             'category',
             'clean_domain',
             'brand_low',
             'judgement_brand',
             'judgement_url',
             'domain_has_negative',
             'organic_has_negative',
             'organic_has_nonEnglish',
             'brand_list',
             'match_brand',
             'match_location_brand',
             'domain_negative',
             'organic_negative',
             'non_english',
             'url_list',
             'match_url',
             'match_location_url',
             'organic_url',
             'search_result',
             'search_result_raw']

In [120]:
### The dictionary to substitute the column name if its a dataframe for the generic domains ###

substitute_dict = {'brand':'product',
                   'brand_low':'product_low',
                   'judgement_brand':'judgement_product',
                   'brand_list':'product_list',
                   'match_brand':'match_product',
                   'match_location_brand':'match_location_product'}

In [19]:
print("Taxonomy name: {}".format(taxonomy_name))

In [122]:
### Get the organized dataframe ###

if 'generic' in taxonomy_name:
    new_column_order = [x for x in column_order if ('url' not in x) and (x != 'category')] # Get columns name for generic domains
    df_final_reorg = df_final[new_column_order] # Reorder the dataframe 
    df_final_reorg = df_final_reorg.rename(columns = substitute_dict) # Change name for columns
else:  # If not indicate the Taxonomy is a Generic list
    try: # Using all the columns
        df_final_reorg = df_final[column_order]
    except KeyError:  # If do not have URL related columns, ignore that
        new_column_order = [x for x in column_order if 'url' not in x]
        df_final_reorg = df_final[new_column_order]

In [20]:
df_final_reorg.to_clipboard(index=0)
print('Done!')